<a href="https://colab.research.google.com/github/moni6264/deeplearning2/blob/main/CNN_PartB_5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PART - B Fine-Tuning a Pre-Trained model**

Import Google colab to drive 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import shutil and splitting the dataset 

In [3]:
import shutil  # provides the ability to copy and removal of files
import os  # provides functions for interacting with the operating system
import random
try:
    shutil.rmtree("inaturalist_12K")
except:
    pass

# Unzip the natural datset     
zip_path = "drive/MyDrive/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip

try:
    os.remove("nature_12K.zip")
except:
    pass
try:
    shutil.rmtree("inaturalist_12K/validation/")
except:
    pass

# dataset directory 
temp = os.listdir("inaturalist_12K/train")
print("Defining class names class names") # print number classes in dataset
class_names = [name for name in temp if name != ".DS_Store"] 

#Spliting valdiation dataset 
validation_split = 0.1 #splitting dataset 10%
def spliting_train_data(validation_split = 0.1):
  os.mkdir("inaturalist_12K/validation/")
  for each_class in class_names:
      os.mkdir("inaturalist_12K/validation/"+each_class)
      train_images = os.listdir("inaturalist_12K/train/"+each_class)
      train_images_per_class = list(train_images)
      print(len(train_images_per_class))
      random.shuffle(train_images_per_class) # Shuffle the training images

      validation_images_per_class = train_images_per_class[:round(validation_split*len(train_images_per_class))]
      print(len(validation_images_per_class))

      for im in validation_images_per_class:
          shutil.move("inaturalist_12K/train/"+each_class+"/"+im, "inaturalist_12K/validation/"+each_class+"/"+im)
spliting_train_data(validation_split)

Defining class names class names
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100


Exploring the Data

In [4]:
train = 0
val = 0
test = 0

for each_class in class_names:
      train_images = os.listdir("inaturalist_12K/train/"+each_class)
      train+=len(list(train_images))

      validation_images = os.listdir("inaturalist_12K/validation/"+each_class)
      
      val+=len(list(validation_images))

      test_images = os.listdir("inaturalist_12K/val/"+each_class)
      
      test+=len(list(test_images))
 # print training validation test      
print("training" , train)
print("validation Examples = ", val)
print("test examples = ", test)

training 9000
validation Examples =  1000
test examples =  2000


Import required libriaries for pre trained module 

In [5]:
import os  
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization


# implement weights and biases 

!pip install wandb
import wandb
from keras.callbacks import EarlyStopping, ModelCheckpoint
from wandb.keras import WandbCallback
from keras.utils.vis_utils import plot_model
from PIL import Image
# image plotting 
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 144 kB 37.9 MB/s 
     |████████████████████████████████| 181 kB 43.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=5b2d690cd865fe2733d11d0f314cb3b28fa2b6031ffc0a4e04c3d7fdc691b758
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


Setup the Train Test and Validation generators


In [6]:
data_dir = "inaturalist_12K"
#data_augmentation = True
# data prepartion 
def data_preparation(data_dir , data_augmentation = True , batch_size = 250):
  train_dir = os.path.join(data_dir, "train")
  val_dir = os.path.join(data_dir, "validation")
  test_dir = os.path.join(data_dir, "val")
  if data_augmentation == True:
    train_datagen = ImageDataGenerator(rescale=1./255,
                                      height_shift_range=0.2,
                                      width_shift_range=0.2,
                                      horizontal_flip=True,
                                      zoom_range=0.2,
                                      fill_mode="nearest")
  else:
      train_datagen = ImageDataGenerator(rescale=1./255)
  val_datagen = ImageDataGenerator(rescale=1./255)
  test_datagen = ImageDataGenerator(rescale=1./255)


  train_generator = train_datagen.flow_from_directory(
          train_dir,
          target_size=(256, 256),
          shuffle=True,
          color_mode="rgb",
          batch_size=batch_size,
          class_mode='categorical')

  val_generator = val_datagen.flow_from_directory(
          val_dir,
          target_size=(256, 256),
          shuffle=True,
          color_mode="rgb",
          batch_size=batch_size,
          class_mode='categorical')
  
  test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size= (256, 256),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode=None)

  
  
  return train_generator , val_generator, test_generator


Implement pre trained InceptionV3, InceptionResNetV2, ResNet50, Xception

In [7]:
def pretrain_model(pretrained_model_name, dropout, dense_layer, pre_layer_train=None):
    
  # keras sequential model based on a pre-trained model intended to be fine tuned
  

    # shape image size 
    shape = (256 ,256, 3)

    # add a pretrained model without the top dense layer
    if pretrained_model_name == 'ResNet50':
      pretrained_model = tf.keras.applications.ResNet50(include_top=False,input_shape=shape, weights='imagenet')
    elif pretrained_model_name == 'InceptionResNetV2':
      pretrained_model = tf.keras.applications.InceptionResNetV2(include_top=False,input_shape= shape, weights='imagenet')
    elif pretrained_model_name == 'InceptionV3':
      pretrained_model = tf.keras.applications.InceptionV3(include_top=False,input_shape= shape, weights='imagenet')    
    elif pretrained_model_name == 'Xception':
      pretrained_model = tf.keras.applications.Xception(include_top=False,input_shape= shape, weights='imagenet')
    else:
      raise Exception('no pretrained model given')
      
    
    #freeze all layers
    for layer in pretrained_model.layers:
        layer.trainable=False 
    
    #setting  top layers as trainable
    if pre_layer_train:
      for layer in pretrained_model.layers[-pre_layer_train:]:
        layer.trainable=True

    model = tf.keras.models.Sequential()
    model.add(pretrained_model)  #adding pretrained model
    model.add(Flatten()) # flatten layer
    model.add(Dense(dense_layer, activation= 'relu'))#adding a dense layer
    model.add(Dropout(dropout)) # dropout
    model.add(Dense(10, activation="softmax"))#softmax layer
# sum model 
    return model

Configuration wandb


In [8]:
pre_train_model = "InceptionV3"   #you can change model for wandb sweeps
def train():
    # Default values for hyper-parameters
    config_defaults = {
        "data_augmentation": True,
        "batch_size": 250,
        "dropout": 0.1,
        "dense_layer": 256,
        "learning_rate": 0.0001,
        "epochs": 5,
        "pre_layer_train": None,
        "pretrained_model_name": 'InceptionV3'
      

    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters 
    config = wandb.config

    # Local variables obtained from wandb config
    data_augmentation = config.data_augmentation
    batch_size = config.batch_size
    dropout = config.dropout
    dense_layer = config.dense_layer
    learning_rate = config.learning_rate
    epochs = config.epochs
    pre_layer_train = config.pre_layer_train
    pretrained_model_name = config.pretrained_model_name
    

    
   # show  the hyperparameters
    run_name = "pre_train_mdl_{}_aug_{}_bs_{}_ep_{}_dropout_{}_dense_{}".format(pre_train_model, data_augmentation, batch_size,epochs, dropout, dense_layer)
    print(run_name)

    #data generators
    train_generator , val_generator, test_generator = data_preparation(data_dir , data_augmentation = True , batch_size = 250)
    
    # calling the pretrained model 
    model = pretrain_model(pretrained_model_name = pretrained_model_name, dropout = dropout, dense_layer = dense_layer, pre_layer_train=pre_layer_train)
    print(model.count_params())

    # compile the model using optmizer  (adam)
    model.compile(optimizer=Adam(learning_rate = learning_rate), loss = 'categorical_crossentropy', metrics = ['accuracy'])

    # Early Stopping method 
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

    # save the best validation accuracy
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)

# model fit 
    history = model.fit(train_generator,
                        steps_per_epoch = train_generator.n//train_generator.batch_size,
                        validation_data = val_generator,
                        validation_steps = val_generator.n//val_generator.batch_size,
                        epochs=epochs, verbose = 2, 
                        callbacks=[WandbCallback(), earlyStopping, checkpoint])
      
    

    wandb.run.name = run_name
    wandb.run.save()
    wandb.run.finish()
    return history

sweeps configuration for hyper parameters 

In [9]:

sweep_config = {
  "name": "DL_Assignment_PartB",
  "metric": {
      "name":"val_accuracy",
      "goal": "maximize"
  },
  "method": "bayes",
  "parameters": {
        "data_augmentation": {
            "values": [True, False]
        },
        "batch_size": {
            "values": [128, 256, 512]
        },
        "learning_rate": {
            "values": [0.001, 0.0001]
        },
        "epochs": {
            "values": [5, 10]
        },
        "dropout": {
            "values": [0, 0.2,0.4]
        },
        "dense_layer": {
            "values": [128, 256, 512]
        },
          "pre_layer_train": {
            "values": [None, 10, 20]
        },
          "pretrained_model_name":{
             "values": ['InceptionResNetV2','ResNet50','InceptionV3','Xception'] 
          },

      
           }
}

sweep_id = "q832m8jh"
wandb.agent(sweep_id,train,entity="moni6264",project="test_part_b",count=5)
#sweep_id = wandb.sweep(sweep_config, entity="moni6264", project="test_part_b")
#wandb.agent(sweep_id, train, count=8)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Agent Starting Run: v6oac0ob with config:
wandb: 	batch_size: 256
wandb: 	data_augmentation: True
wandb: 	dense_layer: 512
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	pre_layer_train: 20
wandb: 	pretrained_model_name: InceptionResNetV2
wandb: Currently logged in as: moni6264 (use `wandb login --relogin` to force relogin)


pre_train_mdl_InceptionV3_aug_True_bs_256_ep_5_dropout_0.2_dense_512
Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
219070464/219055592 [==============================] - 2s 0us/step
82653930
Epoch 1/5
35/35 - 359s - loss: 5.1183 - accuracy: 0.5822 - val_loss: 8.5489 - val_accuracy: 0.4060 - _timestamp: 1648225349.0000 - _runtime: 382.0000 - 359s/epoch - 10s/step
Epoch 2/5
35/35 - 303s - loss: 0.7929 - accuracy: 0.7506 - val_loss: 1.2030 - val_accuracy: 0.7030 - _timestamp: 1648225687.0000 - _runtime: 720.0000 - 303s/epoch - 9s/step
Epoch 3/5
35/35 - 298s - loss: 0.6889 - accuracy: 0.7785 - val_loss: 0.8598 - val_accuracy: 0.7480 - _timestamp: 1648226004.0000 - _runtime: 1037.0000 - 298s/epoch - 9s/step
Epoch 4/5
35/35 - 298s - loss: 0.6311 - accuracy: 0.8012 - val_loss: 0.8125 - val_accuracy: 0.7740 - _timestamp: 1648226302.0000 - _runtime: 1335.0000 - 298s/epoch - 9s/step
Epoch 5/5
35/35 - 284s - loss: 

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


accuracy,▁▆▇██
epoch,▁▃▅▆█
loss,█▁▁▁▁
val_accuracy,▁▇███
val_loss,█▁▁▁▁
accuracy,0.81141
best_epoch,3
best_val_loss,0.81246
epoch,4
loss,0.58362
val_accuracy,0.762


...

pre_train_mdl_InceptionV3_aug_True_bs_128_ep_5_dropout_0.4_dense_512
Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
87975986
Epoch 1/5
35/35 - 293s - loss: 2.8101 - accuracy: 0.2581 - val_loss: 2.2529 - val_accuracy: 0.2960 - _timestamp: 1648230109.0000 - _runtime: 302.0000 - 293s/epoch - 8s/step
Epoch 2/5
35/35 - 288s - loss: 1.3690 - accuracy: 0.5460 - val_loss: 1.1907 - val_accuracy: 0.6180 - _timestamp: 1648230430.0000 - _runtime: 623.0000 - 288s/epoch - 8s/step
Epoch 3/5
35/35 - 287s - loss: 0.9272 - accuracy: 0.6962 - val_loss: 0.9157 - val_accuracy: 0.7020 - _timestamp: 1648230717.0000 - _runtime: 910.0000 - 287s/epoch - 8s/step
Epoch 4/5
35/35 - 285s - loss: 0.7460 - accuracy: 0.7613 - val_loss: 0.9003 - val_accuracy: 0.7170 - _timestamp: 1648231002.0000 - _runtime: 1195.0000 - 285s/epoch - 8s/step
Epoch 5/5
35/35 - 287s - loss: 0.6221 - accuracy: 0.7967 - val_loss: 0.8465 - val_accuracy: 0.7400 -

accuracy,▁▅▇██
epoch,▁▃▅▆█
loss,█▃▂▁▁
val_accuracy,▁▆▇██
val_loss,█▃▁▁▁
accuracy,0.79666
best_epoch,4
best_val_loss,0.84651
epoch,4
loss,0.62212
val_accuracy,0.74


wandb: Agent Starting Run: zarvt9x9 with config:
wandb: 	batch_size: 128
wandb: 	data_augmentation: True
wandb: 	dense_layer: 512
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	pre_layer_train: 20
wandb: 	pretrained_model_name: Xception


pre_train_mdl_InceptionV3_aug_True_bs_128_ep_5_dropout_0.4_dense_512
Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
87975986
Epoch 1/5
35/35 - 298s - loss: 3.0323 - accuracy: 0.3525 - val_loss: 2.3292 - val_accuracy: 0.5330 - _timestamp: 1648231665.0000 - _runtime: 310.0000 - 298s/epoch - 9s/step
Epoch 2/5
35/35 - 286s - loss: 1.2002 - accuracy: 0.6236 - val_loss: 1.5126 - val_accuracy: 0.6560 - _timestamp: 1648231978.0000 - _runtime: 623.0000 - 286s/epoch - 8s/step
Epoch 3/5
35/35 - 289s - loss: 0.8774 - accuracy: 0.7291 - val_loss: 0.9176 - val_accuracy: 0.7300 - _timestamp: 1648232299.0000 - _runtime: 944.0000 - 289s/epoch - 8s/step
Epoch 4/5
35/35 - 289s - loss: 0.7061 - accuracy: 0.7736 - val_loss: 0.8975 - val_accuracy: 0.7390 - _timestamp: 1648232592.0000 - _runtime: 1237.0000 - 289s/epoch - 8s/step
Epoch 5/5
35/35 - 281s - loss: 0.6170 - accuracy: 0.8049 - val_loss: 0.9050 - val_accuracy: 0.7430 -

accuracy,▁▅▇██
epoch,▁▃▅▆█
loss,█▃▂▁▁
val_accuracy,▁▅███
val_loss,█▄▁▁▁
accuracy,0.80489
best_epoch,3
best_val_loss,0.89746
epoch,4
loss,0.61703
val_accuracy,0.743
